In [1]:
import json as json
import numpy as np
import pandas as pd
from pyspark.sql.types import *
from pyspark.sql import Row

In [2]:
%pylab inline
from pyspark.sql.types import * 

Populating the interactive namespace from numpy and matplotlib


In [20]:
from pyspark.sql import SQLContext 
titlesdf = sqlContext.createDataFrame(pd.read_json('nasa_title.json')).persist(StorageLevel.MEMORY_AND_DISK_SER)
titlesdf.registerTempTable('title')

Peek json contents

In [21]:
keywordsdf = sqlContext.createDataFrame(pd.read_json('nasa_keyword.json')).persist(StorageLevel.MEMORY_AND_DISK_SER)
keywordsdf.registerTempTable('keyword')
keywordsdf.show()

+--------------------+--------------------+
|                  id|             keyword|
+--------------------+--------------------+
|55942a57c63a7fe59...|ArrayBuffer(EARTH...|
|55942a57c63a7fe59...|ArrayBuffer(EARTH...|
|55942a58c63a7fe59...|ArrayBuffer(EARTH...|
|55942a58c63a7fe59...|ArrayBuffer(EARTH...|
|55942a58c63a7fe59...|ArrayBuffer(EARTH...|
|55942a58c63a7fe59...|ArrayBuffer(EARTH...|
|55942a58c63a7fe59...|ArrayBuffer(EARTH...|
|55942a58c63a7fe59...|ArrayBuffer(EARTH...|
|55942a58c63a7fe59...|ArrayBuffer(EARTH...|
|55942a58c63a7fe59...|ArrayBuffer(EARTH...|
|55942a58c63a7fe59...|ArrayBuffer(EARTH...|
|55942a58c63a7fe59...|ArrayBuffer(EARTH...|
|55942a58c63a7fe59...|ArrayBuffer(EARTH...|
|55942a58c63a7fe59...|ArrayBuffer(EARTH...|
|55942a58c63a7fe59...|ArrayBuffer(EARTH...|
|55942a58c63a7fe59...|ArrayBuffer(EARTH...|
|55942a58c63a7fe59...|ArrayBuffer(EARTH...|
|55942a58c63a7fe59...|ArrayBuffer(EARTH...|
|55942a58c63a7fe59...|ArrayBuffer(EARTH...|
|55942a58c63a7fe59...|ArrayBuffe

In [22]:
descriptiondf = sqlContext.createDataFrame(pd.read_json('nasa_description.json')).persist(StorageLevel.MEMORY_AND_DISK_SER)
descriptiondf.registerTempTable('description')

In [23]:
landingdf = sqlContext.createDataFrame(pd.read_json('nasa_landingPage.json')).persist(StorageLevel.MEMORY_AND_DISK_SER)
landingdf.registerTempTable('landingPage')

In [24]:
languagedf = sqlContext.createDataFrame(pd.read_json('nasa_language.json')).persist(StorageLevel.MEMORY_AND_DISK_SER)
languagedf.registerTempTable('language')

In [25]:
themedf = sqlContext.createDataFrame(pd.read_json('nasa_theme.json')).persist(StorageLevel.MEMORY_AND_DISK_SER)
themedf.registerTempTable('theme')

In [26]:
temporaldf = sqlContext.createDataFrame(pd.read_json('nasa_temporal.json')).persist(StorageLevel.MEMORY_AND_DISK_SER)
temporaldf.registerTempTable('temporal')

In [28]:
spatialdf = sqlContext.createDataFrame(pd.read_json('nasa_spatial.json')).persist(StorageLevel.MEMORY_AND_DISK_SER)
spatialdf.registerTempTable('spatial')

In [29]:
modifieddf = sqlContext.createDataFrame(pd.read_json('nasa_modified.json')).persist(StorageLevel.MEMORY_AND_DISK_SER)
modifieddf.registerTempTable('modified')

In [46]:
from operator import add

title_count = titlesdf.map( lambda x: x.title.replace(',',' ').replace('.',' ').replace('-',' ').lower()) \
        .flatMap(lambda x: x.split()) \
        .map(lambda x: (x, 1)) \
        .reduceByKey(lambda x,y:x+y) \
        .map(lambda x:(x[1],x[0])) \
        .sortByKey(False) 

In [47]:
output = title_count.collect()  

In [51]:
iter_count = 0
for (count, word) in output:
    print("%i: %s" % (count, word))
    iter_count = iter_count + 1
    if iter_count > 50:
        break

7555: project
4843: and
4344: for
2994: data
2703: of
2394: level
1807: global
1720: 0
1649: v1
1485: 3
1433: 2
1397: daily
1387: 1
1311: l2
1157: in
1142: high
1136: monthly
1120: a
1086: from
1066: surface
1035: the
946: v003
930: product
917: gpm
909: system
893: grid
854: l3
821: boreas
803: v005
797: lba
788: 5
727: ground
683: eco
682: land
674: based
663: grids
630: validation
625: 2000
623: v7
621: swath
607: omi/aura
600: degree
589: rss
588: soil
575: temperature
575: cloud
573: space
562: total
552: v001
548: ozone
546: aerosol


In [61]:
titlesdf.take(5)

[Row(id=u'55942a57c63a7fe59b495a77', title=u'15 Minute Stream Flow Data: USGS (FIFE)'),
 Row(id=u'55942a57c63a7fe59b495a78', title=u'15 Minute Stream Flow Data: USGS (FIFE)'),
 Row(id=u'55942a58c63a7fe59b495a79', title=u'15 Minute Stream Flow Data: USGS (FIFE)'),
 Row(id=u'55942a58c63a7fe59b495a7a', title=u'2000 Pilot Environmental Sustainability Index (ESI)'),
 Row(id=u'55942a58c63a7fe59b495a7b', title=u'2000 Pilot Environmental Sustainability Index (ESI)')]

In [73]:
title_sentences = titlesdf.map(lambda x: ''.join(x.title)) \
        .flatMap(lambda x: x.split('.'))

In [74]:
title_bigram = title_sentences.map(lambda x: x.split()) \
    .flatMap(lambda x: [((x[i],x[i+1]),1) for i in range(0,len(x)-1)])

In [75]:
title_freq_bigrams = title_bigram.reduceByKey(lambda x,y:x+y) \
    .map(lambda x:(x[1],x[0])) \
    .sortByKey(False)

In [134]:
title_freq_bigrams.take(50)

[(836, (u'Level', u'3')),
 (682, (u'L3', u'Global')),
 (550, (u'GROUND', u'VALIDATION')),
 (550, (u'GPM', u'GROUND')),
 (488, (u'L2', u'Swath')),
 (469, (u'NETCDF', u'V7')),
 (460, (u'FROM', u'DMSP')),
 (453, (u'Level', u'2')),
 (448, (u'PRODUCT', u'GRIDS')),
 (448, (u'OCEAN', u'PRODUCT')),
 (444, (u'Daily', u'L3')),
 (367, (u'Land', u'Surface')),
 (357, (u'Total', u'Column')),
 (356, (u'GHRSST', u'Level')),
 (336, (u'AVERAGE', u'FROM')),
 (336, (u'SSM/I', u'OCEAN')),
 (336, (u'RSS', u'SSM/I')),
 (335, (u'Lat/Lon', u'Grid')),
 (324, (u'AIRS/Aqua', u'Level')),
 (324, (u'SAFARI', u'2000')),
 (321, (u'Mixing', u'Ratio')),
 (319, (u'Global', u'0')),
 (306, (u'System', u'Project')),
 (299, (u'Grid', u'V003')),
 (286, (u'of', u'the')),
 (278, (u'Aqua', u'AIRS')),
 (273, (u'25', u'deg')),
 (261, (u'x', u'0')),
 (258, (u'MLS/Aura', u'L2')),
 (249, (u'Applications', u'Project')),
 (245, (u'Surface', u'Model')),
 (243, (u'Systems', u'Project')),
 (242, (u'Model', u'L4')),
 (241, (u'deg', u'Lat/L

In [91]:
desc_sentences = descriptiondf.map(lambda x: ''.join(x.description)) \
        .flatMap(lambda x: x.split('.'))

In [101]:
desc_bigram = desc_sentences.map(lambda x: x.split()) \
    .flatMap(lambda x: [((x[i],x[i+1]),1) for i in range(0,len(x)-1)])

In [131]:
desc_freq_bigrams = desc_bigram.reduceByKey(lambda x,y:x+y) \
    .map(lambda x:(x[1],x[0])) \
    .sortByKey(False)
desc_freq_bigrams.take(50)

[(37194, (u'in', u'the')),
 (24517, (u'of', u'the')),
 (11715, (u'on', u'the')),
 (9989, (u'is', u'a')),
 (9100, (u'from', u'the')),
 (8993, (u'and', u'Aqua')),
 (8979, (u'the', u'equator')),
 (8962, (u'equator', u'in')),
 (8010, (u'for', u'the')),
 (7824, (u'to', u'the')),
 (6696, (u'data', u'set')),
 (6667, (u'will', u'be')),
 (5998, (u'spectral', u'bands,')),
 (5814, (u'development', u'of')),
 (5772, (u'aboard', u'the')),
 (5559, (u'over', u'the')),
 (5484, (u'These', u'data')),
 (5464, (u'by', u'the')),
 (5381, (u'the', u'development')),
 (5161, (u'and', u'the')),
 (5126, (u'and', u'in')),
 (5108, (u'so', u'that')),
 (4994, (u'data', u'in')),
 (4978, (u'the', u'entire')),
 (4888, (u'data', u'will')),
 (4860, (u'able', u'to')),
 (4774, (u'understanding', u'of')),
 (4771, (u'the', u'Earth')),
 (4749, (u'that', u'it')),
 (4744, (u'data', u'are')),
 (4739, (u'of', u'our')),
 (4707, (u'across', u'the')),
 (4697, (u'role', u'in')),
 (4692, (u'instrument', u'aboard')),
 (4681, (u'of', u'g

In [129]:

from nltk.corpus import stopwords
nltk_stop = stopwords.words('english')

In [125]:
def remove_stopwords_bigram(rec_tup):
    if rec_tup[0][0] not in nltk_stop and rec_tup[0][1] not in nltk_stop:
        return rec_tup
    return ((), 0)

desc_bigram_nostopwords = desc_bigram.map(remove_stopwords_bigram)

In [126]:
desc_bigram_nostopwords.take(5)

[((u'USGS', u'15'), 1),
 ((u'15', u'minute'), 1),
 ((u'minute', u'stream'), 1),
 ((u'stream', u'flow'), 1),
 ((u'flow', u'data'), 1)]

In [132]:
desc_freq_bigrams_nostopwords = desc_bigram_nostopwords.reduceByKey(lambda x,y:x+y) \
    .map(lambda x:(x[1],x[0])) \
    .sortByKey(False)

In [133]:
desc_freq_bigrams_nostopwords.take(50)

[(6696, (u'data', u'set')),
 (5998, (u'spectral', u'bands,')),
 (5484, (u'These', u'data')),
 (4692, (u'instrument', u'aboard')),
 (4600, (u'Earth', u'system')),
 (4600, (u'Resolution', u'Imaging')),
 (4564, (u'Moderate', u'Resolution')),
 (4548, (u"Earth's", u'surface')),
 (4514, (u'global', u'change')),
 (4503, (u'system', u'models')),
 (4496, (u'Aqua', u'MODIS')),
 (4493, (u'Imaging', u'Spectroradiometer)')),
 (4493, (u'every', u'1')),
 (4492, (u'acquiring', u'data')),
 (4488, (u'Terra', u'MODIS')),
 (4485, (u'orbit', u'around')),
 (4484, (u'entire', u"Earth's")),
 (4483, (u'vital', u'role')),
 (4482, (u'processes', u'occurring')),
 (4482, (u'lower', u'atmosphere')),
 (4482, (u'key', u'instrument')),
 (4481, (u'Terra', u'(EOS')),
 (4481, (u'days,', u'acquiring')),
 (4481, (u'wavelengths', u'(see')),
 (4481, (u'(see', u'MODIS')),
 (4481, (u'surface', u'every')),
 (4481, (u'(EOS', u'AM)')),
 (4481, (u'passes', u'south')),
 (4481, (u'Aqua', u'passes')),
 (4481, (u'accurately', u'enough

Questions:
1. What are the keywords that are also most frequent words in titles? What are their frequencies?
2. What are the keywords that are also most frequent words in descriptions? What are their frequencies?
3. Can we use word frequencies in title to accurately predict keywords?
4. What is the TF/IDF for each word in description?
5. Can we use TF/IDF in description to accurately predict keywords?
6. How do you use bigrams/n-grams? Jaccard-similarity (or other similarity measure) between n-grams in title and n-grams in description? To accurately predict the keywords of other title-description pairs? Use splitting test & train?